In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
#sys.path.append('/home/jma819/post_cmfe_analysis')
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as utils_jjm
import dlc_utils
import caiman
import statsmodels.formula.api as smf
import scipy.spatial.distance as dist
import itertools
import math
import warnings
import numbers
warnings.filterwarnings(action='once')

/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.incremental_pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of 

In [82]:
reload(utils_jjm)
reload(dlc_utils)

<module 'dlc_utils' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/dlc_utils.py'>

In [3]:
cnmfe_file_key = pd.read_csv('/volumes/My_Passport/dlc_analysis/behavcamvideos/cnmfe_file_key.csv')
#cnmfe_file_key = pd.read_csv('/projects/p30771/dlc_analysis/openfield_dlc_output/cnmfe_file_key.csv')

In [4]:
cnmfe_base_dir = '/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/'
#cnmfe_base_dir = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
wt_CNMFE_file = ['30-Mar_20_39_05_out.mat', '30-Mar_20_45_16_out.mat', '27-Feb_17_33_59_out.mat', '22-Mar_22_52_02_out.mat',
                 '28-Feb_16_10_05_out.mat', '27-Feb_17_32_15_out.mat', '28-Feb_16_21_21_out.mat', '25-Mar_13_27_27_out.mat',
                 '25-Mar_14_22_02_out.mat', '25-Mar_14_22_44_out.mat', '26-Mar_18_33_55_out.mat', '27-Mar_00_26_12_out.mat', '27-Mar_00_48_46_out.mat']

ko_CNMFE_files = ['31-Mar_13_28_15_out.mat', '29-Mar_21_42_20_out.mat', '13-Apr_17_57_40_out.mat', '29-Mar_14_27_55_out.mat', '13-Apr_16_01_20_out.mat',
                 '13-Apr_16_11_27_out.mat', '29-Mar_13_39_44_out.mat']

In [5]:
tracking_files = ['/volumes/My_Passport/dlc_analysis/behavcamvideos/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 
#tracking_files = ['/projects/p30771/dlc_analysis/openfield_dlc_output/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 

In [6]:
CNMFE_dir_paths_list = [str(cnmfe_base_dir+fname) for fname in wt_CNMFE_file]
#CNMFE_dir_paths_list

In [7]:
binning_time = 1 # seconds 
body_part_for_tracking = 'tail_base' #
number_of_bins = 50 #
polynomial_degree = 2 #

grouped_raw_data = {}
success = []
failed = []
for CNMFE_file, tracking_file in zip(CNMFE_dir_paths_list, tracking_files):
    #print(CNMFE_file)
    #print(tracking_file)
    try:
        # load cell fluorescence 
        cell_fluorescence = sio.loadmat(CNMFE_file)
        C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
        C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
        # create tracking time deltas
        interpolated = utils_jjm.prepare_timedelta_dfs(tracking_file)
        #load spatial components by session
        com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6)
        cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components)
        #C_z_scored_filtered = utils_jjm.filter_out_by_size(C_z_scored, cell_contours, for_dims, 0.6, 100)
        #store results 
        grouped_raw_data[tracking_file.split('/')[-2]] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 'C_normalized_z_scored': C_normalized_z_scored, 
                                                          'interpolated' : interpolated, 'com' : com_df, 'spatial_components' : spatial_components, 'cell_contours': cell_contours,  
                                                         'for_dims' : for_dims}
        success.append((tracking_file.split('/')[-2], CNMFE_file.split('/')[-1]))
    except FileNotFoundError:
        failed.append(tracking_file)
    except OSError:
        failed.append(tracking_file)

## spatial clustering

In [8]:
## combine dfs for each session to bin velocity values across dfs
#downsample
new_sampling_interval = .2
V_df = pd.concat([grouped_raw_data[session]['interpolated'].resample(str(new_sampling_interval)+'S').max() 
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))
all_sessions_v_bins = pd.cut(V_df['tail_base'], bins=50)

V_df['velocity_bins'] = all_sessions_v_bins

In [9]:
# compile and filter fluorescence
filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
    filtered_for_analysis[session] = utils_jjm.filter_out_by_size(grouped_raw_data[session]['C_normalized_z_scored'], grouped_raw_data[session]['cell_contours'], 
                                                                  grouped_raw_data[session]['for_dims'], 0.6, 100)
C_df = pd.concat([filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))


In [83]:
# clustering info per session 
coactivity_dfs = {}
for session in tqdm(list(grouped_raw_data.keys())):
    # get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    #compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)

    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score threshold
    cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
    binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

    reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

    #ks_results_2sided, ks_one_sided_more, ks_one_sided_less, coactivity_dfs = utils_jjm.spatial_coordination_by_session(reindexed, 10, com_distances)
    coactivity_dfs[session] = utils_jjm.spatial_coordination_by_session(reindexed)
    
    



  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [12:32<2:30:31, 752.62s/it]

 15%|█▌        | 2/13 [33:32<2:45:51, 904.71s/it]

 23%|██▎       | 3/13 [1:28:07<4:29:20, 1616.03s/it]

 31%|███       | 4/13 [1:37:02<3:13:43, 1291.52s/it]

 38%|███▊      | 5/13 [3:24:22<6:18:09, 2836.24s/it]

 46%|████▌     | 6/13 [5:56:35<9:11:16, 4725.17s/it]

 54%|█████▍    | 7/13 [8:13:33<9:37:17, 5772.97s/it]

 62%|██████▏   | 8/13 [8:38:31<6:14:13, 4490.64s/it]

 69%|██████▉   | 9/13 [8:53:02<3:46:58, 3404.72s/it]

 77%|███████▋  | 10/13 [9:08:38<2:13:11, 2663.98s/it]

 85%|████████▍ | 11/13 [10:52:23<2:04:24, 3732.25s/it]

 92%|█████████▏| 12/13 [12:06:16<1:05:42, 3942.52s/it]

100%|██████████| 13/13 [13:31:02<00:00, 3743.26s/it]  


In [24]:
len(reindexed)

6250

In [96]:
len(coactivity_dfs['GRIN013_H13_M33_S54'])

6250

In [126]:
np.shape(list(coactivity_dfs['GRIN013_H13_M33_S54'][0].values))

(1, 630)

In [102]:
np.shape(list(coactivity_dfs['GRIN013_H13_M33_S54'][0].columns))

(630, 2)

In [106]:
#list(coactivity_dfs['GRIN013_H13_M33_S54'][0].columns)

In [125]:
#save coactivity info to h5py file
for session in tqdm(list(grouped_raw_data.keys())):
    f = h5py.File("/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/openfield_analysis/spatial_clustering/" + str(session) +"clustering.h5", "w")

    for time_pt in range(len(coactivity_dfs[session])):
        dset = f.create_dataset(str(time_pt), np.shape(coactivity_dfs[session][time_pt].values), dtype=np.float32)
        dset[:, :] = coactivity_dfs[session][time_pt].values

    cell_pair_ids = f.create_dataset('cell_ids', np.shape(list(coactivity_dfs[session][0].columns)), dtype='i')
    cell_pair_ids[:, :] = list(coactivity_dfs[session][0].columns)

    f.close()



  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:06<01:18,  6.53s/it]

 15%|█▌        | 2/13 [00:17<01:25,  7.81s/it]

 23%|██▎       | 3/13 [00:29<01:31,  9.17s/it]

 31%|███       | 4/13 [00:39<01:23,  9.24s/it]

 38%|███▊      | 5/13 [00:59<01:41, 12.71s/it]

 46%|████▌     | 6/13 [01:29<02:05, 17.93s/it]

 54%|█████▍    | 7/13 [01:54<01:58, 19.80s/it]

 62%|██████▏   | 8/13 [02:08<01:31, 18.29s/it]

 69%|██████▉   | 9/13 [02:23<01:08, 17.21s/it]

 77%|███████▋  | 10/13 [02:32<00:43, 14.58s/it]

 85%|████████▍ | 11/13 [02:49<00:31, 15.53s/it]

 92%|█████████▏| 12/13 [03:03<00:15, 15.03s/it]

100%|██████████| 13/13 [03:26<00:00, 15.90s/it]


In [113]:
f['70'][0,4]

1.0

In [117]:
cell_pair_ids[4, 0]

1

In [79]:
session='GRIN013_H13_M33_S54'

cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)

com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score thresholds
cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

In [80]:
reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

In [81]:
outs = utils_jjm.spatial_coordination_by_session(reindexed)


  2%|▏         | 152/6250 [00:20<12:51,  7.90it/s]


  5%|▍         | 303/6250 [00:40<13:56,  7.11it/s]


  5%|▌         | 337/6250 [00:44<12:51,  7.67it/s]

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/abc.py", line 180, in __instancecheck__
    def __instancecheck__(cls, instance):
KeyboardInterrupt

  8%|▊         | 483/6250 [01:03<12:00,  8.00it/s]


 10%|█         | 634/6250 [01:22<12:25,  7.54it/s]


 13%|█▎        | 785/6250 [01:42<12:09,  7.49it/s]


 13%|█▎        | 817/6250 [01:47<11:53,  7.62it/s]

KeyboardInterrupt: 

In [42]:
outs

[           1                              ... 33          34       35    36
            2  3  4  6  7  8  9  10 11 12  ... 34 35 36 37 35 36 37 36 37 37
 coactivity  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
 
 [1 rows x 630 columns]]

In [50]:
reindexed

,1,2,3,4,6,7,8,9,10,11,...,28,29,30,31,32,33,34,35,36,37
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6245,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6246,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
#make following code work with these variables for function
binnums = 10

In [ ]:
ks_results_2sided = []
ks_one_sided_more = []
ks_one_sided_less = []

time_points = tqdm(range(len(reindexed)))
for time_index in time_points:
    coactivity_by_time_point = {}
    for pair in itertools.combinations(list(binned_fluorescence.columns), 2):
        if (reindexed.loc[time_index][pair[0]] == 1) and (reindexed.loc[time_index][pair[1]] == 1):
            coactivity_by_time_point[pair] = 1
        else:
            coactivity_by_time_point[pair] = 0
    coactivity_df = pd.DataFrame(coactivity_by_time_point, index=['coactivity'])

    coactive_cell_distances = pairwise_distance[coactivity_df[coactivity_df == 1].dropna(axis=1).columns]
    non_coactive_distances = pairwise_distance[coactivity_df[coactivity_df == 0].dropna(axis=1).columns]

    #linear_distribution = np.linspace(1, len(coactive_cell_distances.columns), binnums)

    #binnums=10
    cum_results_coactive = stats.cumfreq(coactive_cell_distances.values[0], numbins=binnums, defaultreallimits=(0, 500))
    cum_results_non_coactive = stats.cumfreq(non_coactive_distances.values[0], numbins=binnums, defaultreallimits=(0, 500))

    #plt.plot(np.linspace(0, 500, binnums), cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), linear_distribution/len(coactive_cell_distances.values[0]))
    #plt.show()

    less_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), 'norm', alternative='less')
    more_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), 'norm', alternative='more')
    ks_one_sided_more.append(more_result)
    ks_one_sided_less.append(less_result)
    
    #coordination_index = utils_jjm.create_coordination_index(more_result, less_result)

    #two sided test between coactive and noncoactive distribution 
    ks_result = stats.ks_2samp(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #coordination_indicies.append(coordination_index)
    ks_results_2sided.append(ks_result)
    

In [ ]:
## for plotting average spatial coordination during trigger regions

In [ ]:

triggered_activity_across_sessions = {}
for session in list(grouped_raw_data.keys()):
    #inputs activity_threshold, resting_threshold, resting_baseline 
    crossing_indicies = utils_jjm.select_trigger_regions(binned_velocity_df[session], 0.5, 0.5, 20)
    #inputs time_to_plot
    threshold_activity = utils_jjm.average_triggered_regions(V_df.loc[session]['tail_base'].values, crossing_indicies, 40)
    
    f_threshold_activity = utils_jjm.average_triggered_regions(spatial_coordination_activity_in_session 
            , crossing_indicies, 40)
    triggered_activity_across_sessions[session] = pd.concat([threshold_activity, f_threshold_activity], axis=1, keys=['velocity', 'spatial_coordination'])

In [ ]:
#triggered_activity_across_sessions

In [ ]:
#concat by mouse 
concacted_by_mouse = {}
for mouse in list(set([session[0:7] for session in list(triggered_activity_across_sessions.keys())])):
    dfs_by_mouse = []
    sessions = []
    for session in list(triggered_activity_across_sessions.keys()):
        if mouse in session:
            if not(triggered_activity_across_sessions[session].empty):
                means = triggered_activity_across_sessions[session].mean(axis=1, level=0)
                dfs_by_mouse.append(means)
                sessions.append(session)
    if len(dfs_by_mouse)>0:
        concacted_by_mouse[mouse] = pd.concat(dfs_by_mouse, axis=1, keys=sessions)
combined_by_mouse = pd.concat(list(concacted_by_mouse.values()), axis=1, keys=list(concacted_by_mouse.keys()))

In [ ]:
combined_by_mouse.head()

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean=combined_by_mouse.mean(axis=1, level=2)['velocity'].values
plt.plot(x, combined_by_mouse.mean(axis=1, level=2)['velocity'].values, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['velocity'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=(0), linestyle='--', color='r')
plt.show()

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean = combined_by_mouse.mean(axis=1, level=2)['fluorescence'].values
plt.plot(x, mean, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['fluorescence'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=0, linestyle='--', color='r')
plt.show()